In [1]:
import numpy as np
import pandas as pd
import matplotlib as plt
import urllib.request
from pulp import *
from scipy import optimize
import math

%matplotlib inline

In [2]:
t = pd.read_csv('../csv/stochastic_matrix.csv', index_col=0).fillna(0)

In [3]:
v = t.T[[
    'R00200', 'R00268', 'R00341', 'R00342', 'R00351',
    'R00405', 'R00658', 'R00703', 'R01015', 'R01049',
    'R01056', 'R01061', 'R01070', 'R01082', 'R01196', 
    'R01197', 'R01325', 'R01512', 'R01518', 'R01528',
    'R01529', 'R01641', 'R01786', 'R01830', 'R01899',
    'R01900', 'R02035', 'R02164', 'R02740', 'R04779'
]].T

for i in v.columns:
    if sum(v[i] == 0) == v.shape[0]:
        v = v.drop(i, axis=1)
        
v['C00007'] = pd.Series(np.zeros(v.shape[0]), index=v.index)
v['C00033'] = pd.Series(np.zeros(v.shape[0]), index=v.index)
v['C00092'] = pd.Series(np.zeros(v.shape[0]), index=v.index)
v['C00014'] = pd.Series(np.zeros(v.shape[0]), index=v.index)
v['C00025'] = pd.Series(np.zeros(v.shape[0]), index=v.index)

In [4]:
df = v.T
df['Glucose_input'] =  pd.Series(np.zeros(df.shape[0]), index=df.index)
df['Glucose_input']['C00267'] = 1

df['Oxygen_input'] =  pd.Series(np.zeros(df.shape[0]), index=df.index)
df['Oxygen_input']['C00007'] = 1

df['CO2_output'] =  pd.Series(np.zeros(df.shape[0]), index=df.index)
df['CO2_output']['C00011'] = -1

df['Lactate_output'] =  pd.Series(np.zeros(df.shape[0]), index=df.index)
df['Lactate_output']['C00186'] = -1

df['Acetate_output'] =  pd.Series(np.zeros(df.shape[0]), index=df.index)
df['Acetate_output']['C00033'] = -1

df['Glutamate_input'] =  pd.Series(np.zeros(df.shape[0]), index=df.index)
df['Glutamate_input']['C00025'] = 1

df['Ammonia_input'] =  pd.Series(np.zeros(df.shape[0]), index=df.index)
df['Ammonia_input']['C00014'] = 1

df['Glu_Exchange'] =  pd.Series(np.zeros(df.shape[0]), index=df.index)
df['Glu_Exchange']['C00267'] = -1
df['Glu_Exchange']['C00092'] = 1

df['acetyl_CoA_hydrolase'] =  pd.Series(np.zeros(df.shape[0]), index=df.index)
df['acetyl_CoA_hydrolase']['C00024'] = -1
df['acetyl_CoA_hydrolase']['C00001'] = -1
df['acetyl_CoA_hydrolase']['C00010'] = 1
df['acetyl_CoA_hydrolase']['C00033'] = 1

df['NADP_Exchange'] =  pd.Series(np.zeros(df.shape[0]), index=df.index)
df['NADP_Exchange']['C00005'] = -1
df['NADP_Exchange']['C00003'] = -1
df['NADP_Exchange']['C00006'] = 1
df['NADP_Exchange']['C00004'] = 1

df['ADP_Exchange'] =  pd.Series(np.zeros(df.shape[0]), index=df.index)
df['ADP_Exchange']['C00002'] = -1
df['ADP_Exchange']['C00008'] = 1

df['AMP_Exchange'] =  pd.Series(np.zeros(df.shape[0]), index=df.index)
df['AMP_Exchange']['C00002'] = -1
df['AMP_Exchange']['C00020'] = -1
df['AMP_Exchange']['C00008'] = 2

df['R00835'] =  pd.Series(np.zeros(df.shape[0]), index=df.index)
df['R00835']['C00092'] = -1
df['R00835']['C00006'] = -1
df['R00835']['C01236'] = 1
df['R00835']['C00005'] = 1
df['R00835']['C00080'] = 1

df['R08575'] =  pd.Series(np.zeros(df.shape[0]), index=df.index)
df['R08575']['C05382'] = -1
df['R08575']['C00118'] = -1
df['R08575']['C00279'] = 1
df['R08575']['C00085'] = 1

df['R02164']['C00004'] = -1
df['R02164']['C00003'] = 1
df['R02164']['C15602'] = 0
df['R02164']['C15603'] = 0

df['R00243'] =  pd.Series(np.zeros(df.shape[0]), index=df.index)
df['R00243']['C00025'] = -1
df['R00243']['C00003'] = -1
df['R00243']['C00001'] = -1

df['R00243']['C00026'] = 1
df['R00243']['C00014'] = 1
df['R00243']['C00004'] = 1
df['R00243']['C00080'] = 1

df['Oxophos'] =  pd.Series(np.zeros(df.shape[0]), index=df.index)
df['Oxophos']['C00004'] = -2
df['Oxophos']['C00007'] = -1
df['Oxophos']['C00008'] = -5
df['Oxophos']['C00002'] = 5
df['Oxophos']['C00003'] = 2
df['Oxophos']['C00011'] = 1

In [5]:
BIOMASS = pd.read_csv('../csv/amino_acid.csv',index_col=0)
df_Biomass =  pd.Series(np.zeros(df.shape[0]), index=df.index)

for i in BIOMASS.index:
    df_Biomass[BIOMASS.T[i]['C_num']] = BIOMASS.T[i]['flux']
    
df_Biomass['C00668'] = -0.27519
df_Biomass['C00119'] = (-0.039036) + (-0.036117) + (-0.053446)

In [6]:
df_new = pd.concat([df_Biomass.T, df], axis=1)
df_new = df_new.rename(columns={0: 'BIOMASS'})
df_new = df_new.drop([
    'C00001','C00009','C00080','C00138','C00139','C15602','C15603', 
#     'C00231'
])

In [9]:
def fba(glc):
    x = []
    for i in df_new.columns:
        if (
                (i=='Oxophos') or (i=='R01899') or (i=='R00268') or (i=='R10343') or (i=='R02164') or (i=='R00835') or 
                (i=='R02035') or (i=='ATP_Exchange') or (i=='AMP_Exchange') or (i=='CO2_output')
        ):
            x.append(LpVariable(i, lowBound=0,upBound=10000, cat='Continuous'))

        elif (i=='Lactate_output'):
            x.append(LpVariable(i, lowBound=0, upBound=10000, cat='Continuous'))
            
        elif (i=='BIOMASS'):
            x.append(LpVariable(i, lowBound=0, upBound=1000, cat='Continuous'))
        
        elif (i=='Oxygen_input'):
            x.append(LpVariable(i, lowBound=100, upBound=100, cat='Continuous'))
        
        elif (i=='Glucose_input'):
            x.append(LpVariable(i, lowBound=glc, upBound=glc, cat='Continuous'))

        elif (i=='Glutamate_input'):
            x.append(LpVariable(i, lowBound=0, upBound=100, cat='Continuous'))
            
        elif (i=='Ammonia_input'):
            x.append(LpVariable(i, lowBound=-10000, upBound=10000, cat='Continuous'))
              
        elif (
            (i=='Succinate_output') or (i=='Malate_output') or (i=='Fumarate_output') or (i=='Acetate_output')
        ):
            x.append(LpVariable(i, lowBound=0, upBound=50, cat='Continuous'))
                
        elif (i=='R00200') or (i=='R01196') or (i=='R01197') or (i=='R00405'):
            x.append(LpVariable(i, lowBound=-10000, upBound=0, cat='Continuous'))

        else:
            x.append(LpVariable(i, lowBound=-10000, upBound=10000, cat='Continuous'))

    m = LpProblem(sense=LpMaximize)
    m += x[0]

    for i in np.dot(df_new, x):
        m += i == 0

    status = m.solve()
    if (LpStatus[status]) == 'Infeasible':
        print(i)
        return 100, 100

    y = []
    for i in x:
        y.append(i.value())

    j = pd.DataFrame(y, index=[str(i) for i in x]).T
    
    return j.T

In [10]:
res = pd.DataFrame()
for i in range(0, 2000):
    res = pd.concat([res, fba(i)], axis=1)

In [13]:
res

,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
BIOMASS,0.0,8.223566,16.447132,24.670698,24.782923,24.782923,24.782923,24.782923,24.782923,24.782923,...,24.782923,24.782923,24.782923,24.782923,24.782923,24.782923,24.782923,24.782923,24.782923,24.782923
R00200,-100.0,-70.975693,-41.951386,-12.927080,-12.530989,-12.530989,-12.530989,-12.530989,-12.530989,-12.530989,...,-3923.271900,-3925.271900,-3927.271900,-3929.271900,-3931.271900,-3933.271900,-3935.271900,-3937.271900,-3939.271900,-3941.271900
R00268,100.0,66.817612,33.635223,0.452835,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
R00341,100.0,82.601895,65.203790,47.805685,45.595550,43.595550,41.595550,39.595550,37.595550,35.595550,...,-15.663551,-15.663551,-15.663551,-15.663551,-15.663551,-15.663551,-15.663551,-15.663551,-15.663551,-15.663551
R00342,200.0,157.571450,115.142890,72.714336,70.162614,68.162614,66.162614,64.162614,62.162614,60.162614,...,8.903513,8.903513,8.903513,8.903513,8.903513,8.903513,8.903513,8.903513,8.903513,8.903513
R00351,-100.0,-66.817612,-33.635223,-0.452835,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
R00405,-200.0,-154.617050,-109.234090,-63.851141,-61.259101,-59.259101,-57.259101,-55.259101,-53.259101,-51.259101,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
R00658,0.0,-11.626202,-23.252404,-34.878606,-33.064560,-31.064560,-29.064560,-27.064560,-25.064560,-23.064560,...,3938.935400,3940.935400,3942.935400,3944.935400,3946.935400,3948.935400,3950.935400,3952.935400,3954.935400,3956.935400
R00703,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,-3910.740900,-3912.740900,-3914.740900,-3916.740900,-3918.740900,-3920.740900,-3922.740900,-3924.740900,-3926.740900,-3928.740900
R01015,0.0,1.615557,3.231114,4.846672,3.882366,2.882366,1.882366,0.882366,-0.117634,-1.117634,...,-1982.117600,-1983.117600,-1984.117600,-1985.117600,-1986.117600,-1987.117600,-1988.117600,-1989.117600,-1990.117600,-1991.117600


In [ ]:
# rate[[
#     'BIOMASS', 'Glucose_input', 'Oxygen_input', 'Glutamate_input', 'Lactate_output'
# ]].plot()
# plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0, fontsize=10)

In [ ]:
rate['BIOMASS'].plot()

In [ ]:
rate[[
    'CO2_output', 'Lactate_output', 
#     'Succinate_output', 
#     'Malate_output', 'Fumarate_output', 'Acetate_output', 
    'Ammonia_output'
]].plot()
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0, fontsize=10)

In [ ]:
rate['Oxygen_input'].plot()

In [ ]:
rate[[
    'Lactate_output', 'Ammonia_output', 'Glucose_input', 'Glutamate_input'
]]

In [ ]:
# norm_result.to_csv('../result/grad_glc.csv')

In [ ]:
# rate.to_csv('../result/grad_glc_all.csv')